In [1]:
import numpy as np
import pandas as pd
from simple.ohlc import renko, midPrice
from simple.plotly import chartFigure
from simple.funcs import symlog
from simple.backtest import npBacktestIOC

In [2]:
T = np.load('data/ETHUSDT.2022-05-08.tick.npz')['ETHUSDT'].view(np.recarray)
len(T)

95820

In [3]:
Buy = T[T.VolumeA > 0]
Sell = T[T.VolumeA < 0]
Buy[:5]

rec.array([('2022-05-08T00:01:00.335000', 0, 2636.,  6.),
           ('2022-05-08T00:01:00.335000', 1, 2636.,  4.),
           ('2022-05-08T00:01:00.668000', 2, 2636., 96.),
           ('2022-05-08T00:01:00.668000', 3, 2636.,  2.),
           ('2022-05-08T00:01:00.668000', 4, 2636.,  2.)],
          dtype={'names':['DateTimeA','LocalTimeA','PriceA','VolumeA'], 'formats':['<M8[us]','<i8','<f8','<f8'], 'offsets':[0,8,16,24], 'itemsize':40})

In [4]:
S = np.load('data/ETHUSDT.2022-05-08.bidask.npz')['ETHUSDT'].view(np.recarray)
S[:5]

rec.array([('2022-05-08T00:01:00.007000', 2636., 2635.99),
           ('2022-05-08T00:01:00.043000', 2636., 2635.99),
           ('2022-05-08T00:01:00.079000', 2636., 2635.99),
           ('2022-05-08T00:01:00.117000', 2636., 2635.99),
           ('2022-05-08T00:01:00.149000', 2636., 2635.99)],
          dtype={'names':['DT','A','B'], 'formats':['<M8[us]','<f4','<f4'], 'offsets':[0,8,16], 'itemsize':24})

In [5]:
len(T), len(S)

(95820, 99941)

In [ ]:
C = renko(T, 15)
C[:5]

In [ ]:
len(C)

In [ ]:
delay = np.timedelta64(10, 'ms')

In [ ]:
signal = 0
signalA = np.zeros(len(S.DT))

for c in C:
    color = 'green' if c[3] > c[2] else 'red'
    duration = T.DateTimeA[c[1]] - T.DateTimeA[c[0]]
    tm = T.DateTimeA[c[1]] + delay
    k = np.searchsorted(S.DT, tm)
    if duration < np.timedelta64(10000, 'μs'):
        if c[3] > c[2]:
            signal = 1
        elif c[3] < c[2]:
            signal = -1
    if k < len(signalA):
        signalA[k] = signal
        
    print(f'{str(c): <26}', T.DateTimeA[c[1]], tm, f'{duration: <16}', k, signal, color, sep='\t')

In [ ]:
trades = npBacktestIOC(S.DT.astype(np.int64), S.A, S.B, signalA, 0)
trades

In [ ]:
Long = trades[trades.Size > 0]
Short = trades[trades.Size < 0]

In [ ]:
# timed chart
fig = chartFigure(rows=2,
    Sell=dict(x=Sell.DateTimeA, y=Sell.PriceA, color='red', mode='markers', size=-symlog(Sell.VolumeA)+3, opacity=0.2),
    Buy=dict(x=Buy.DateTimeA, y=Buy.PriceA, color='green', mode='markers', size=symlog(Buy.VolumeA)+3, opacity=0.2),
    Ask=dict(x=S.DT, y=S.A, color='red', opacity=0.2, shape='hv', mode='lines'),
    Bid=dict(x=S.DT, y=S.B, color='green', opacity=0.2, shape='hv', mode='lines'),

    EnterLong=dict(x=Long.T0, y=Long.Price0, mode='markers',
               marker=dict(symbol='triangle-up', size=12, line_color='darkgreen', line_width=1, color='green')),
    ExitLong=dict(x=Long.T1, y=Long.Price1, mode='markers',
               marker=dict(symbol='x', size=10, line_color='darkgreen', line_width=1, color='green')),

    EnterShort=dict(x=Short.T0, y=Short.Price0, mode='markers',
               marker=dict(symbol='triangle-down', size=12, line_color='darkred', line_width=1, color='red')),
    ExitShort=dict(x=Short.T1, y=Short.Price1, mode='markers', 
               marker=dict(symbol='x', size=10, line_color='darkred', line_width=1, color='red')),
    Signal=dict(x=S.DT, y=signalA, row=2, color='magenta', shape='hv')
)
fig

In [13]:
fig.layout.shapes = [{
     'fillcolor': 'green' if c[3] > c[2] else 'red',
     'line': {'color': 'black', 'width': 2},
     'opacity': 0.15,
     'type': 'rect',
     'x0': str(T.DateTimeA[int(c[0])]),
     'x1': str(T.DateTimeA[int(c[1])]),
     'y0': c[2],
     'y1': c[3]
 } for c in C]